In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [17]:
adult = pd.read_csv("adult.csv")
adult.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [18]:
X = adult.drop("income", axis=1)
y = adult["income"]

In [19]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [20]:
y.map({"<=50K": 0, ">50K": 1})

0        0
1        0
2        1
3        1
4        0
        ..
48837    0
48838    1
48839    0
48840    0
48841    1
Name: income, Length: 48842, dtype: int64

In [21]:
col_one_hot, col_label = [], []
for col in X.columns:
    if X[col].dtype == "object":
        if len(X[col].unique()) <= 10:
            col_one_hot.append(col)
        else:
            col_label.append(col)

In [22]:
col_one_hot

['workclass', 'marital-status', 'relationship', 'race', 'gender']

In [23]:
col_label

['education', 'occupation', 'native-country']

In [24]:
for i in col_one_hot:
    print(len(X[i].unique()))

9
7
6
5
2


In [25]:
for i in col_label:
    print(len(X[i].unique()))

16
15
42


In [26]:
le = LabelEncoder()
for col in col_label:
    X[col] = le.fit_transform(X[col])

In [27]:
X = pd.get_dummies(X, columns=col_one_hot)

In [28]:
X.head()

,age,fnlwgt,education,educational-num,occupation,capital-gain,capital-loss,hours-per-week,native-country,workclass_?,...,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,gender_Female,gender_Male
0,25,226802,1,7,7,0,0,40,39,0,...,1,0,0,0,0,1,0,0,0,1
1,38,89814,11,9,5,0,0,50,39,0,...,0,0,0,0,0,0,0,1,0,1
2,28,336951,7,12,11,0,0,40,39,0,...,0,0,0,0,0,0,0,1,0,1
3,44,160323,15,10,7,7688,0,40,39,0,...,0,0,0,0,0,1,0,0,0,1
4,18,103497,15,10,0,0,0,30,39,1,...,1,0,0,0,0,0,0,1,1,0


In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [30]:
lr = LogisticRegression(max_iter=5000)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

/home/shahroz/anaconda3/envs/deep-learning/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8320196540075749

In [31]:
y_pred = lr.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.8320196540075749
              precision    recall  f1-score   support

       <=50K       0.86      0.93      0.89      7479
        >50K       0.69      0.52      0.59      2290

    accuracy                           0.83      9769
   macro avg       0.78      0.72      0.74      9769
weighted avg       0.82      0.83      0.82      9769

[[6934  545]
 [1096 1194]]
